# TACO: Generation Pipeline

In this notebook we aim to test the TACO pipeline, using a single example to generate code and make the evaluation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import polars as pl
import torch
import numpy as np

## LLM
from src.llms import Llama3_1_Instruct

seed = 42
# NumPy
np.random.seed(seed)

# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of GPUs available: 1
GPU 0: NVIDIA RTX A5000


## Load Data

In [2]:
PATH = "../../data/TACO/processed"
train_input = pl.read_ipc(f"{PATH}/train.feather")
train_evaluation = pl.read_ipc(f"{PATH}/train_evaluation_tests.feather")
train_solutions = pl.read_ipc(f"{PATH}/train_solutions.feather")


In [3]:
## ID 4 is easy
ID = 4
## Get the input string
input_example = train_input.filter(pl.col("id") == ID).select("input").unique().to_numpy().squeeze(1)[0]

In [4]:
llm = Llama3_1_Instruct()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.81s/it]


In [ ]:
NUM_SEQUENCES=20
outputs = []
for i in range(20//NUM_SEQUENCES):

    config = {
        "temperature": 0.7,
        "max_length": 2048,
        "top_p": 0.95,
        "num_return_sequences": NUM_SEQUENCES
    }

    prompt = f"\nQUESTION: \n{input_example} \n Please write a Python program."
    output = llm.run(prompt=prompt, input=input_example, config_params=config)

    for res in output:
        outputs.append(res)

In [15]:
import json
json.dump(outputs, open(f"outputs.json", "w"))

## Parser

In [16]:
generations = []
for o in outputs:
    generations.append(o["generated_text"])

In [17]:
count = 0
for g in generations:
    if g.count("python") >= 1:
        count += 1
count

14

In [24]:
import ast
import re
gens = []

for i in range(len(generations)):

    code_blocks = re.findall(r'```python(.*?)```', generations[i], re.DOTALL)
    extracted_code = "\n".join([block.strip() for block in code_blocks])
    gens.append(extracted_code)
    



In [26]:
def generate_results_json(outputs: list, prompt: str, id: id):

    results = {}

    results["task_id"] = id
    results["prompt"] = prompt
    results["output"] = outputs


    json.dump([results], open(f"results.json", "w"))

In [27]:
generate_results_json(gens, prompt, ID)